### Imports

In [1]:
# Run once in terminal (not inside notebook)
# pip install -r requirements.txt

import os
from dotenv import load_dotenv

# LangChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# LangGraph
from langgraph.graph import StateGraph, END

# CrewAI
from crewai import Agent, Task, Crew

d:\All_Projects\Prayatna\venv\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.10) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
d:\All_Projects\Prayatna\venv\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.10) which Google will stop supporting in new releases of google.cloud.resourcemanager_v3 once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.cloud.resourcemanager_v3 past that date.
  warnings.warn(message, FutureWarning)


### Load API keys & LLM

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1")
 
load_dotenv()

llm = ChatOpenAI(
    model="openai/gpt-oss-20b:free",
    temperature=0,
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_API_BASE,
)

### Langchain Explanation prompt

In [3]:
explain_prompt = PromptTemplate(
    input_variables=["case_data"],
    template="""
You are a government fraud detection analyst.

Analyze the following welfare beneficiary data and explain why the case is suspicious.

Case Data:
{case_data}

Provide:
- Bullet-point reasoning
- Mention duplicate identity, shared financial links, or abnormal registrations
- Keep explanation concise and professional
"""
)

explanation_chain = LLMChain(
    llm=llm,
    prompt=explain_prompt
)


C:\Users\Dell\AppData\Local\Temp\ipykernel_33224\1588665569.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  explanation_chain = LLMChain(


### LangGraph reasoning workflow

In [4]:
def explain_node(state: dict) -> dict:
    explanation = explanation_chain.run(case_data=state["case"])
    return {"explanation": explanation}


workflow = StateGraph(dict)
workflow.add_node("explain_case", explain_node)
workflow.set_entry_point("explain_case")
workflow.add_edge("explain_case", END)

graph = workflow.compile()

### CrewAI multi-agent setup

In [ ]:
fraud_analyst = Agent(
    role="Fraud Analyst",
    goal="Detect suspicious welfare fraud patterns",
    backstory="Expert in government welfare fraud analytics.",
)

auditor = Agent(
    role="Government Auditor",
    goal="Summarize fraud evidence clearly",
    backstory="Responsible for compliance and investigation reports.",
)

investigation_task = Task(
    description="""
Investigate the suspicious welfare case and produce a final explanation
based on duplicate Aadhaar, shared bank accounts, and abnormal registrations.
""",
    expected_output="A detailed explanation of why the case is suspicious, including key indicators.",
    agent=fraud_analyst,
)

audit_task = Task(
    description="""
Create a concise audit-ready explanation of why the case is suspicious.
""",
    expected_output="A concise summary of the fraud evidence for audit purposes.",
    agent=auditor,
)

crew = Crew(
    agents=[fraud_analyst, auditor],
    tasks=[investigation_task, audit_task],
    llm=llm,
    verbose=True,
)

NameError: name 'fraud_analyst' is not defined

### Run suspicious case example

In [6]:
suspicious_case = {
    "aadhaar_like_id": "4587 9214 6632",
    "phone_number": "9876543210",
    "bank_account": "123456789012",
    "registrations_per_aadhaar": 3,
    "bank_shared_count": 5,
    "phone_shared_count": 4,
    "agent_cluster_size": 18,
    "annual_income": "₹85,000",
    "district": "Bhopal"
}

# LangGraph reasoning
result = graph.invoke({"case": suspicious_case})

print("\n--- AI Explanation (LangGraph) ---\n")
print(result["explanation"])

# CrewAI investigation
print("\n--- CrewAI Investigation ---\n")
crew.kickoff()

C:\Users\Dell\AppData\Local\Temp\ipykernel_33224\440860251.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  explanation = explanation_chain.run(case_data=state["case"])



--- AI Explanation (LangGraph) ---

**Suspicious Indicators**

- **Multiple registrations per Aadhaar‑like ID** – 3 separate welfare accounts linked to the same identifier, far above the typical single‑beneficiary threshold.
- **Shared bank account** – The bank account “123456789012” is used by 5 different Aadhaar‑like IDs, indicating a common financial hub that could be a fraud ring.
- **Shared phone number** – The phone “9876543210” is associated with 4 distinct Aadhaar‑like IDs, suggesting a single contact point for multiple beneficiaries.
- **Large agent cluster** – 18 agents are connected to this cluster, which is unusually high for a single district (Bhopal) and may point to coordinated manipulation.
- **Low reported income** – Annual income of ₹85,000 is significantly below the average for the district, yet the beneficiary has multiple registrations and shared financial links, raising red flags for income misreporting or benefit inflation.

**Conclusion**  
The combination of d

NameError: name 'crew' is not defined